In [6]:
def calculate(expression: str) -> str:
    return str(eval(expression))


def get_employee_details(emp_id: int) -> dict:
    employees = {
        101: {"name": "Akshay", "department": "GenAI"},
        102: {"name": "Ravi", "department": "Backend"},
    }
    return employees.get(emp_id, "Employee not found")


In [7]:
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser

llm = ChatOllama(model="llama3")

tool_prompt = PromptTemplate(
    input_variables=["query"],
    template="""
You are an AI assistant.

Decide which tool to use:
- If math → respond with: TOOL:calculate:<expression>
- If employee lookup → respond with: TOOL:employee:<emp_id>
- Otherwise respond normally.

User query: {query}
"""
)


In [8]:
decision_chain = tool_prompt | llm | StrOutputParser()

decision_chain.invoke("What is 20 * 5 + 10?")


'TOOL:calculate:20*5+10'

In [9]:
def run_tool(response: str):
    if response.startswith("TOOL:calculate:"):
        expr = response.split("TOOL:calculate:")[1]
        return calculate(expr)

    if response.startswith("TOOL:employee:"):
        emp_id = int(response.split("TOOL:employee:")[1])
        return get_employee_details(emp_id)

    return response


In [10]:
query = "Get employee details for emp id 101"

decision = decision_chain.invoke(query)
result = run_tool(decision)

result


{'name': 'Akshay', 'department': 'GenAI'}

In [11]:
query = "What is 20 * 5 + 10?"

decision = decision_chain.invoke(query)
result = run_tool(decision)

result


'110'